In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv("Political-media-DFE.csv",encoding_errors='ignore')

In [4]:
data = df[["_unit_id","text","label","audience", "message", "embed","bias", "bias:confidence"]]
data.head()

,_unit_id,text,label,audience,message,embed,bias,bias:confidence
0,766192484,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...,From: Trey Radel (Representative from Florida),national,policy,"<blockquote class=""twitter-tweet"" width=""450"">...",partisan,1.0
1,766192485,VIDEO - #Obamacare: Full of Higher Costs and ...,From: Mitch McConnell (Senator from Kentucky),national,attack,"<blockquote class=""twitter-tweet"" width=""450"">...",partisan,1.0
2,766192486,Please join me today in remembering our fallen...,From: Kurt Schrader (Representative from Oregon),national,support,"<blockquote class=""twitter-tweet"" width=""450"">...",neutral,1.0
3,766192487,RT @SenatorLeahy: 1st step toward Senate debat...,From: Michael Crapo (Senator from Idaho),national,policy,"<blockquote class=""twitter-tweet"" width=""450"">...",neutral,1.0
4,766192488,.@amazon delivery #drones show need to update ...,From: Mark Udall (Senator from Colorado),national,policy,"<blockquote class=""twitter-tweet"" width=""450"">...",partisan,1.0


In [3]:
label_to_id = {label: i for i, label in enumerate(df['bias'].unique())}
id_to_label = {i: label for label, i in label_to_id.items()}
df['bias'] = df['bias'].map(label_to_id)
df['bias']

0       0
1       0
2       1
3       1
4       0
       ..
4995    0
4996    0
4997    1
4998    1
4999    1
Name: bias, Length: 5000, dtype: int64

In [9]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

class BiasDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return {
            'text': self.texts[idx],
            'label': self.labels[idx]
        }

def train(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader):
        optimizer.zero_grad()
        inputs = tokenizer(batch['text'], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
        labels = torch.tensor(batch['label']).to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(data_loader):
            inputs = tokenizer(batch['text'], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
            labels = batch['label']
            outputs = model(**inputs)
#             preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            probabilities = torch.sigmoid(outputs.logits)
            preds = torch.argmax((probabilities > 0.5).long(), dim=1).cpu().numpy()
#             print(preds)
            all_preds.extend(preds)
            all_labels.extend(labels)
    return accuracy_score(all_labels, all_preds)

# Load dataset
df = data
df = df.dropna(subset=['text', 'bias'])  # Drop rows with missing values in 'text' or 'bias'

# Convert bias labels to numerical format
label_to_id = {label: i for i, label in enumerate(df['bias'].unique())} # "partisan" = 0, "neutral" = 1
id_to_label = {i: label for label, i in label_to_id.items()}
df['bias'] = df['bias'].map(label_to_id)

# Split dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['bias'], test_size=0.2, random_state=42)

# Create DataLoader
train_dataset = BiasDataset(train_texts.tolist(), train_labels.tolist())
test_dataset = BiasDataset(test_texts.tolist(), test_labels.tolist())
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Load pre-trained model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id)).to(device)

# Training settings
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)

#Training loop
epochs = 10
for epoch in range(epochs):
    train_loss = train(model, train_loader, optimizer, device)
    print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}')

    test_accuracy = evaluate(model, test_loader, device)
    print(f'Epoch {epoch + 1}/{epochs}, Test Accuracy: {test_accuracy:.4f}')

# train_loss = train(model, train_loader, optimizer, device)
# test_accuracy = evaluate(model, test_loader, device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_96/1827561085.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).to(device)
100%|██████████| 500/500 [00:56<00:00,  8.78it/s]


Epoch 1/10, Train Loss: 0.5468


100%|██████████| 125/125 [00:04<00:00, 28.67it/s]


Epoch 1/10, Test Accuracy: 0.7830


  0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_96/1827561085.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).to(device)
100%|██████████| 500/500 [00:58<00:00,  8.61it/s]


...

Epoch 9/10, Test Accuracy: 0.7690


  0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_96/1827561085.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).to(device)
100%|██████████| 500/500 [00:58<00:00,  8.53it/s]


Epoch 10/10, Train Loss: 0.3229


100%|██████████| 125/125 [00:04<00:00, 28.11it/s]

Epoch 10/10, Test Accuracy: 0.7640


In [5]:
df2 = pd.read_csv("politifact.csv",encoding_errors='ignore')

In [6]:
df2 =df2[['author','statement','source','target']]
df2.head()

,author,statement,source,target
0,Ciara O'Rourke,"“The airport in Salzburg, Austria, has a count...",Instagram posts,false
1,Ciara O'Rourke,Video shows Palestinians pretending to be corp...,Viral image,false
2,Loreben Tuquero,The life span of a wind tower generator lasts ...,Facebook posts,false
3,Ciara O'Rourke,Matthew Perry died because of a COVID-19 vaccine.,Instagram posts,false
4,Jill Terreri,A discrepancy in the number of ballots and vot...,New York Citizens Audit,false


In [7]:
df3 = pd.read_csv("train.tsv",sep='\t', header=None)

In [8]:
df3 = df3[[1,2,4,13]]
df3.head()

,1,2,4,13
0,false,Says the Annies List political group supports ...,dwayne-bohac,a mailer
1,half-true,When did the decline of coal start? It started...,scott-surovell,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",barack-obama,Denver
3,false,Health care reform legislation is likely to ma...,blog-posting,a news release
4,half-true,The economic turnaround started at the end of ...,charlie-crist,an interview on CNN


In [9]:
frame_temp = pd.DataFrame({'author': df3[4], 'statement': df3[2], 'source': df3[13], 'target': df3[1]})
frame_temp

,author,statement,source,target
0,dwayne-bohac,Says the Annies List political group supports ...,a mailer,false
1,scott-surovell,When did the decline of coal start? It started...,a floor speech.,half-true
2,barack-obama,"Hillary Clinton agrees with John McCain ""by vo...",Denver,mostly-true
3,blog-posting,Health care reform legislation is likely to ma...,a news release,false
4,charlie-crist,The economic turnaround started at the end of ...,an interview on CNN,half-true
...,...,...,...,...
10235,aclu-florida,There are a larger number of shark attacks in ...,"interview on ""The Colbert Report""",mostly-true
10236,alan-powell,Democrats have now become the party of the [At...,an interview,mostly-true
10237,herman-cain,Says an alternative to Social Security that op...,a Republican presidential debate,half-true
10238,jeff-greene,On lifting the U.S. Cuban embargo and allowing...,a televised debate on Miami's WPLG-10 against ...,false


In [10]:
data2 = pd.concat([df2,frame_temp], axis=0)
data2

,author,statement,source,target
0,Ciara O'Rourke,"“The airport in Salzburg, Austria, has a count...",Instagram posts,false
1,Ciara O'Rourke,Video shows Palestinians pretending to be corp...,Viral image,false
2,Loreben Tuquero,The life span of a wind tower generator lasts ...,Facebook posts,false
3,Ciara O'Rourke,Matthew Perry died because of a COVID-19 vaccine.,Instagram posts,false
4,Jill Terreri,A discrepancy in the number of ballots and vot...,New York Citizens Audit,false
...,...,...,...,...
10235,aclu-florida,There are a larger number of shark attacks in ...,"interview on ""The Colbert Report""",mostly-true
10236,alan-powell,Democrats have now become the party of the [At...,an interview,mostly-true
10237,herman-cain,Says an alternative to Social Security that op...,a Republican presidential debate,half-true
10238,jeff-greene,On lifting the U.S. Cuban embargo and allowing...,a televised debate on Miami's WPLG-10 against ...,false


In [11]:
data2.groupby('target').count()['author']

target
barely-true    5306
false          8938
full-flop       190
half-flip        82
half-true      5848
mostly-true    5394
no-flip          28
pants-fire     3942
true           4270
Name: author, dtype: int64

In [12]:
data2.head()

,author,statement,source,target
0,Ciara O'Rourke,"“The airport in Salzburg, Austria, has a count...",Instagram posts,false
1,Ciara O'Rourke,Video shows Palestinians pretending to be corp...,Viral image,false
2,Loreben Tuquero,The life span of a wind tower generator lasts ...,Facebook posts,false
3,Ciara O'Rourke,Matthew Perry died because of a COVID-19 vaccine.,Instagram posts,false
4,Jill Terreri,A discrepancy in the number of ballots and vot...,New York Citizens Audit,false


In [23]:
x=np.array(data2[['author','statement']])
x

array([["Ciara O'Rourke",
        '“The airport in Salzburg, Austria, has a counter for people who flew to Austria instead of Australia.”'],
       ["Ciara O'Rourke",
        'Video shows Palestinians pretending to be corpses amid the Israel-Hamas war.'],
       ['Loreben Tuquero',
        'The life span of a wind tower generator lasts just three to four years.'],
       ...,
       ['herman-cain',
        'Says an alternative to Social Security that operates in Galveston County, Texas, has meant that participants will retire with a whole lot more money than under Social Security.'],
       ['jeff-greene',
        'On lifting the U.S. Cuban embargo and allowing travel to Cuba.'],
       ['michael-steele',
        "The Department of Veterans Affairs has a manual out there telling our veterans stuff like, 'Are you really of value to your community?' You know, encouraging them to commit suicide."]],
      dtype=object)

In [16]:
# Using BERT on New Liar Liar dataset
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
from tqdm import tqdm
import pandas as pd


class LiarDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return {
            'statement': self.texts[idx],
            'target': self.labels[idx]
        }

def train(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader):
        optimizer.zero_grad()
        inputs = tokenizer(batch['statement'], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
        labels = torch.tensor(batch['target'], dtype=torch.long).to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(data_loader):
            inputs = tokenizer(batch['statement'], padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
            labels = batch['target']
            outputs = model(**inputs)
#             preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
#             preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            # Use softmax to convert logits to probabilities
            probabilities = F.softmax(outputs.logits, dim=1)
            # Use argmax to get the predicted class
            preds = torch.argmax(probabilities, dim=1).cpu().numpy()
#             print(preds)
            all_preds.extend(preds)
            print(all_preds)
            all_labels.extend(labels)
            print(all_labels)
    return accuracy_score(all_labels, all_preds)

# Load dataset
df = data2[:5000]
df = df.dropna(subset=['statement', 'target'])  # Drop rows with missing values in 'text' or 'bias'

# Convert bias labels to numerical format
label_to_id = {label: i for i, label in enumerate(df['target'].unique())} # "partisan" = 0, "neutral" = 1
id_to_label = {i: label for label, i in label_to_id.items()}
df['target'] = df['target'].map(label_to_id)

# Split dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['statement'], df['target'], test_size=0.2, random_state=42)

# Create DataLoader
train_dataset = LiarDataset(train_texts.tolist(), train_labels.tolist())
test_dataset = LiarDataset(test_texts.tolist(), test_labels.tolist())
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Load pre-trained model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id)).to(device)

# Training settings
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)

#Training loop
epochs = 3
for epoch in range(epochs):
    train_loss = train(model, train_loader, optimizer, device)
    print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}')

    test_accuracy = evaluate(model, test_loader, device)
    print(f'Epoch {epoch + 1}/{epochs}, Test Accuracy: {test_accuracy:.4f}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_111/4234155973.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['target'], dtype=torch.long).to(device)
100%|██████████| 500/500 [47:23<00:00,  5.69s/it]


Epoch 1/3, Train Loss: 1.8648


  1%|          | 1/125 [00:00<01:53,  1.09it/s]

[0, 0, 0, 0, 0, 0, 0, 0]
[tensor(0), tensor(3), tensor(2), tensor(2), tensor(0), tensor(0), tensor(0), tensor(1)]


  2%|▏         | 2/125 [00:01<01:57,  1.04it/s]

...

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 125/125 [02:07<00:00,  1.02s/it]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

  0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_111/4234155973.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['target'], dtype=torch.long).to(device)
 74%|███████▍  | 370/500 [28:52<10:08,  4.68s/it]


KeyboardInterrupt: 

In [64]:
label_to_id = {label: i for i, label in enumerate(data2[:5000]['target'].unique())}
label_to_id

{'false': 0,
 'half-true': 1,
 'barely-true': 2,
 'pants-fire': 3,
 'mostly-true': 4,
 'true': 5,
 'full-flop': 6,
 'half-flip': 7,
 'no-flip': 8}

In [65]:
id_to_label = {i: label for label, i in label_to_id.items()}
id_to_label

{0: 'false',
 1: 'half-true',
 2: 'barely-true',
 3: 'pants-fire',
 4: 'mostly-true',
 5: 'true',
 6: 'full-flop',
 7: 'half-flip',
 8: 'no-flip'}

## Allside Data Scarping

In [72]:
import pandas as pd
import os
import re
import requests

from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import date

urls = ["https://www.allsides.com/unbiased-balanced-news", "https://www.allsides.com/topics/coronavirus?search=coronavirus",
       "https://www.allsides.com/topics/media-bias?search=media%20bias", 
       "https://www.allsides.com/topics/2024-presidential-election?search=2024%20presidential%20election", 
       "https://www.allsides.com/topics/economy-and-jobs?search=economy%20and%20jobs",
       "https://www.allsides.com/topics/abortion?search=abortion",
       'https://www.allsides.com/topics/criminal-justice?search=criminal%20justice',
       'https://www.allsides.com/topics/education?search=education', 
       "https://www.allsides.com/topics/environment?search=environment",
       "https://www.allsides.com/topics/healthcare?search=healthcare",
       "https://www.allsides.com/topics/bridging-divides?search=bridging%20divides",
       "https://www.allsides.com/topics/free-speech?search=free%20speech",
       "https://www.allsides.com/topics/gun-control-and-gun-rights?search=gun%20control%20and%20gun%20rights",
       "https://www.allsides.com/topics/immigration?search=immigration",
       "https://www.allsides.com/topics/inequality?search=inequality",
       "https://www.allsides.com/topics/lgbtq-issues?search=lgbtq%20issues",
       "https://www.allsides.com/topics/middle-east?search=middle%20east",
       "https://www.allsides.com/topics/polarization?search=polarization",
       "https://www.allsides.com/topics/race?search=race%20and%20racism",
       "https://www.allsides.com/topics/religion-and-faith?search=religion%20and%20faith",
       "https://www.allsides.com/topics/taxes?search=taxes",
       "https://www.allsides.com/topics/trade?search=trade"]

In [42]:
def scrape_allsides_cat(URL):
    web = requests.get(URL)
    soup = BeautifulSoup(web.text, 'html.parser')
    trio = soup.find('div', attrs={'class': 'news-trio'})
    left = trio.find_all('div', attrs={'class':'news-item left'})
    left_urls.extend([l.find('a').get('href') for l in left])

    center = trio.find_all('div', attrs={'class':'news-item center'})
    center_urls.extend([c.find('a').get('href') for c in center])

    right = trio.find_all('div', attrs={'class':'news-item right'})
    right_urls.extend([r.find('a').get('href') for r in right])

In [159]:
# topic: coronavirus
# scrape_allsides_cat("https://www.allsides.com/topics/coronavirus?search=coronavirus")

In [172]:
# topic: media bias
# scrape_allsides_cat("https://www.allsides.com/topics/media-bias?search=media%20bias")

# topic: 2024 Presidential Election
# scrape_allsides_cat("https://www.allsides.com/topics/2024-presidential-election?search=2024%20presidential%20election")

# topic: Economy and Jobs
# scrape_allsides_cat("https://www.allsides.com/topics/economy-and-jobs?search=economy%20and%20jobs")

In [97]:
def extract_article_text(url):
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')

    # scraping headers
    if soup.find_all("h1") == []:       # if no header
        article_text = "NA"
        headers= "NA"
        read_more_link= "NA"
        page_tags= "NA"
    else:
        for header in soup.find_all("h1"):
            headers = header.get_text()
    #     headers = [header.get_text() for header in soup.find_all("h1")]

        # scraping text
        article_text = ' '.join(p.get_text() for p in soup.find_all('p'))

        # Extracting the 'Read more' link
        read_more_div = soup.find('div', class_='read-more-story')
        read_more_link = read_more_div.find('a')['href'] if read_more_div else 'No link found'

        # Extracting page tags
        page_tags_div = soup.find('div', class_='page-tags')
        page_tags = [a.get_text() for a in page_tags_div.find_all('a')] if page_tags_div else []
    
    return article_text, headers, read_more_link, page_tags

# Example usage:
# article_url = 'https://www.allsides.com/news/2023-11-09-1237/religion-and-faith-theologians-call-christian-churches-acknowledge-address'
# text = extract_article_text(article_url)
# print(text)

In [66]:
extract_article_text('https://www.allsides.com/news/2023-11-16-2217/general-news-adam-johnson-s-girlfriend-made-heartbreaking-discovery-after-his')

(' As Adam Johnson’s loved ones continue to pick up the pieces following his tragic death last month, new details have come to light about the future the former NHL player envisioned with his longtime girlfriend, Ryan Wolfe. Johnson, who died at the age of 29 after his neck was cut by the skate blade of an opponent in an Elite Ice Hockey League game in England on Oct. 28, had planned to propose to Wolfe, two friends of his family recently shared with USA Today. The late Nottingham Panthers forward... November 16th, 2023 November 16th, 2023 Andy Gorel November 16th, 2023 AllSides Staff November 16th, 2023',
 'Adam Johnson’s girlfriend made heartbreaking discovery after his death',
 'https://nypost.com/2023/11/16/sports/adam-johnsons-girlfriend-made-heartbreaking-discovery-after-his-death/',
 ['General News', 'NHL', 'Death', 'Sports'])

In [73]:
left_urls = []
center_urls = []
right_urls = []
for link in urls:
    scrape_allsides_cat(link)

In [74]:
len(left_urls)

337

In [91]:
response = requests.get('https://www.allsides.com/node/888216')
html = response.content
soup = BeautifulSoup(html, 'html.parser')
# for header in soup.find_all("h1"):
#     headers = header.get_text()
    
# headers
soup.find_all("h1") == []

True

In [75]:
left_content = {}
for link in left_urls:
    url = link
    text1, header1, readmore_link1, tags1 = extract_article_text(url)
    left_content[url, header1] = text1, readmore_link1, tags1

In [98]:
right_content = {}
for link in right_urls:
    url = link
    print(url)
    text2, header2, readmore_link2, tags2 = extract_article_text(url)
    right_content[url, header2] = text2, readmore_link2, tags2

https://www.allsides.com/news/2023-11-16-2221/middle-east-israel-reportedly-considering-hostage-deal-idf-strikes-home-hamas
https://www.allsides.com/news/2023-11-16-2217/general-news-adam-johnson-s-girlfriend-made-heartbreaking-discovery-after-his
https://www.allsides.com/news/2023-11-16-1711/joe-biden-doj-not-expected-charge-biden-classified-docs
https://www.allsides.com/news/2023-11-16-1647/middle-east-hamas-s-war-women
https://www.allsides.com/news/2023-11-16-1411/violence-america-fbi-s-wray-warns-hamas-other-islamist-terror-groups-launching
https://www.allsides.com/news/2023-11-16-1410/business-1b-fake-designer-goods-seized-largest-bust-us-history
https://www.allsides.com/news/2023-11-16-1407/2024-presidential-election-biden-says-gavin-newsom-could-have-job-president
https://www.allsides.com/news/2023-11-16-1401/world-un-official-causes-uproar-over-claim-israel-doesnt-have-right-defend
https://www.allsides.com/news/2023-11-16-0821/2024-presidential-election-nikki-haley-pledges-defu

https://www.allsides.com/news/2023-11-03-1905/economy-and-jobs-how-daylight-saving-time-clock-changes-upend-economy
https://www.allsides.com/news/2023-11-01-1303/economy-and-jobs-fraud-claims-swamp-colleges-biden-administration-expands
https://www.allsides.com/news/2023-10-31-1611/economy-and-jobs-why-halloween-candy-so-expensive-sugar-protectionism
https://www.allsides.com/news/2023-10-31-1110/economy-and-jobs-what-great-trucking-recession-warning-us-about-economy
https://www.allsides.com/news/2023-10-30-2042/banking-and-finance-goldman-sachs-no-longer-sees-us-government-shutdown-2023
https://www.allsides.com/news/2023-10-26-1552/economy-and-jobs-strong-growth-data-no-soft-landing-inflation
https://www.allsides.com/news/2023-10-24-1559/economy-and-jobs-jpmorgan-ceo-jamie-dimon-central-banks-have-been-dead-wrong
https://www.allsides.com/news/2023-10-24-0952/economy-and-jobs-markets-are-sounding-alarm-deficits
https://www.allsides.com/news/2023-11-09-1522/politics-virginia-gop-casts-bla

https://www.allsides.com/news/2023-08-01-1312/public-health-cases-leprosy-historically-uncommon-us-surge-central-florida-cdc
https://www.allsides.com/news/2023-08-01-0609/healthcare-first-alzheimers-disease-risk-assessment-blood-test-consumers-hits
https://www.allsides.com/news/2023-07-18-0716/healthcare-illegal-immigrant-kids-tuberculosis-infections-released-44-states
https://www.allsides.com/news/2023-06-23-0752/healthcare-national-warrior-call-day-proposed-bipartisan-bill-fight-veteran
https://www.allsides.com/news/2023-06-12-1516/joe-biden-biden-gets-two-day-root-canal-after-report-president-eats-child
https://www.allsides.com/news/2023-06-02-0649/federal-state-and-tribal-powers-south-carolina-chips-away-nanny-state
https://www.allsides.com/news/2023-05-30-1052/criminal-justice-theranos-fraudster-elizabeth-holmes-start-11-year-prison
https://www.allsides.com/news/2023-05-23-1536/public-health-next-pandemic-even-deadlier-covid-coming-warns-who
https://www.allsides.com/news/2023-05-1

https://www.allsides.com/news/2023-08-25-0628/housing-and-homelessness-san-francisco-mayor-slams-homeless-coalition-after
https://www.allsides.com/news/2023-08-18-0348/federal-state-and-tribal-powers-us-launches-program-provide-more-native
https://www.allsides.com/news/2023-07-14-0703/housing-and-homelessness-declaring-right-housing-wont-solve-homelessness
https://www.allsides.com/news/2023-05-15-0959/education-desantis-signs-legislation-eliminating-diversity-equity-and-inclusion
https://www.allsides.com/news/2023-05-05-1158/education-colleges-expand-segregated-graduation-events
https://www.allsides.com/news/2023-04-20-0753/climate-change-push-eliminate-fossil-fuels-hurting-poor-people
https://www.allsides.com/news/2023-02-16-1559/joe-biden-biden-signs-racial-equity-order-requiring-federal-agencies-build
https://www.allsides.com/news/2023-01-16-1354/inequality-joe-manchin-traveling-world-economic-forum-davos-part-us-delegation
https://www.allsides.com/news/2023-01-12-0840/inequality-de

https://www.allsides.com/news/2023-07-28-0835/facts-and-fact-checking-liberal-commentators-go-all-kamala-harris-s-lie-about
https://www.allsides.com/news/2023-07-28-0709/race-and-racism-no-silver-lining-slavery-tim-scott-goes-desantis-over-new
https://www.allsides.com/news/2023-07-24-0931/media-industry-cnn-commentator-rips-kamala-harris-over-claim-about-florida
https://www.allsides.com/news/2023-07-21-0550/education-florida-does-not-teach-students-slavery-was-good
https://www.allsides.com/news/2023-07-19-1311/middle-east-israels-herzog-sharply-addresses-criticism-house-members-after
https://www.allsides.com/news/2023-11-16-1647/middle-east-hamas-s-war-women
https://www.allsides.com/news/2023-11-09-1237/religion-and-faith-theologians-call-christian-churches-acknowledge-address
https://www.allsides.com/news/2023-11-10-0720/religion-and-faith-pro-life-ministry-run-sisters-life-nuns-wins-victory-against
https://www.allsides.com/news/2023-11-06-1337/lgbtq-issues-christian-wedding-photograp

In [99]:
center_content = {}
for link in center_urls:
    url = link
    text3, header3, readmore_link3, tags3 = extract_article_text(url)
    center_content[url, header3] = text3, readmore_link3, tags3

In [100]:
import csv

# def save_to_csv(data, filename):
#     with open(filename, 'w', newline='', encoding='utf-8') as file:
#         writer = csv.writer(file)
        
#         if data = "left_content"
#         # Write the header
#         writer.writerow(['URL', 'Header', 'Content'])
#         # Write the data
#         for (url, header), content in data.items():
#             # Writing only a snippet of content for demonstration
#             writer.writerow([url, header, content])
            
def save_to_csv(left_content, center_content, right_content, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(['URL', 'Header', 'Content', 'Read More Link', 'Page Tags', 'Label'])

        # Function to format page tags as a string
        def format_tags(tags):
            return ', '.join(tags)

        # Write the data with orientation labels
        for (url, header), (content, read_more_link, page_tags) in left_content.items():
            writer.writerow([url, header, content, read_more_link, format_tags(page_tags), 'Left'])
        for (url, header), (content, read_more_link, page_tags) in center_content.items():
            writer.writerow([url, header, content, read_more_link, format_tags(page_tags), 'Center'])
        for (url, header), (content, read_more_link, page_tags) in right_content.items():
            writer.writerow([url, header, content, read_more_link, format_tags(page_tags), 'Right'])

# Example usage:
save_to_csv(left_content, center_content, right_content, 'allside_articles.csv')

In [181]:
def append_to_csv(left_content, center_content, right_content, filename):
    with open(filename, 'a', newline='', encoding='utf-8') as file:  # 'a' is for append mode
        writer = csv.writer(file)
            
        for (url, header), content in left_content.items():
            writer.writerow([url, header, content, 'Left'])
        for (url, header), content in center_content.items():
            writer.writerow([url, header, content, 'Center'])
        for (url, header), content in right_content.items():
            writer.writerow([url, header, content, 'Right'])

# Example usage to append data:
append_to_csv(left_content, center_content, right_content, 'allside_articles.csv')  # Assuming new_data is a dictionary with the new left-oriented articles


In [104]:
allside = pd.read_csv("allside_articles.csv")
allside

,URL,Header,Content,Read More Link,Page Tags,Label
0,https://www.allsides.com/news/2023-11-16-2138/...,"How Santos Spent Donors’ Money: Ferragamo, Onl...",George Santos was utterly triumphant. He had ...,https://www.nytimes.com/2023/11/16/nyregion/sa...,"Politics, George Santos, Pornography, House Et...",Left
1,https://www.allsides.com/news/2023-11-16-2132/...,The Guardian Deletes Osama Bin Laden Letter Af...,The Guardian has taken down a letter written ...,https://www.huffpost.com/entry/osama-bin-laden...,"Media Industry, News Literacy, TikTok, The Gua...",Left
2,https://www.allsides.com/news/2023-11-16-1738/...,Supreme Court Refuses to Revive Florida Law Re...,The Supreme Court refused on Thursday to revi...,https://www.nytimes.com/2023/11/16/us/politics...,"Supreme Court, Drag Queens, LGBTQ Issues",Left
3,https://www.allsides.com/news/2023-11-16-1707/...,Mistrial declared in federal case against form...,A mistrial was declared Thursday by a judge i...,https://abcnews.go.com/US/mistrial-declared-fe...,"Justice, Breonna Taylor, Police, Criminal Just...",Left
4,https://www.allsides.com/news/2023-11-16-1653/...,GOP rivals try to cancel Nikki Haley over soci...,Republican presidential candidate Nikki Haley...,https://www.msnbc.com/the-reidout/reidout-blog...,"2024 Presidential Election, Nikki Haley, Socia...",Left
...,...,...,...,...,...,...
884,https://www.allsides.com/news/2022-12-06-0928/...,Biden Starts a Climate Trade War,Wasn’t President Biden going to end Donald Tr...,https://www.wsj.com/articles/biden-starts-a-cl...,"Economy And Jobs, Trade, World, Joe Biden, Cli...",Right
885,https://www.allsides.com/news/2022-12-01-1427/...,India Will Surpass Japan And Germany As The Wo...,India is projected to surpass Japan and Germa...,https://www.dailywire.com/news/india-will-surp...,"World, Economy And Jobs, India, Japan, Germany...",Right
886,https://www.allsides.com/news/2022-10-11-0629/...,"Nissan pulls out of Russia, sells all assets t...",The Nissan car manufacturer sold all of its a...,https://www.foxbusiness.com/economy/nissan-pul...,"Russia, World, Ukraine War, Manufacturing, Bus...",Right
887,https://www.allsides.com/news/2022-09-27-0851/...,Home heating prices are surging. Here's how th...,Natural gas home heating in New England will ...,https://www.washingtonexaminer.com/restoring-a...,"Energy, Natural Gas, Weather, Housing And Home...",Right


# Political Affiliation Bias Detection

In [9]:
import pandas as pd

# Load the dataset
file_path = 'allside_articles.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,URL,Header,Content,Read More Link,Page Tags,Label
0,https://www.allsides.com/news/2023-11-16-2138/...,"How Santos Spent Donors’ Money: Ferragamo, Onl...",George Santos was utterly triumphant. He had ...,https://www.nytimes.com/2023/11/16/nyregion/sa...,"Politics, George Santos, Pornography, House Et...",Left
1,https://www.allsides.com/news/2023-11-16-2132/...,The Guardian Deletes Osama Bin Laden Letter Af...,The Guardian has taken down a letter written ...,https://www.huffpost.com/entry/osama-bin-laden...,"Media Industry, News Literacy, TikTok, The Gua...",Left
2,https://www.allsides.com/news/2023-11-16-1738/...,Supreme Court Refuses to Revive Florida Law Re...,The Supreme Court refused on Thursday to revi...,https://www.nytimes.com/2023/11/16/us/politics...,"Supreme Court, Drag Queens, LGBTQ Issues",Left
3,https://www.allsides.com/news/2023-11-16-1707/...,Mistrial declared in federal case against form...,A mistrial was declared Thursday by a judge i...,https://abcnews.go.com/US/mistrial-declared-fe...,"Justice, Breonna Taylor, Police, Criminal Just...",Left
4,https://www.allsides.com/news/2023-11-16-1653/...,GOP rivals try to cancel Nikki Haley over soci...,Republican presidential candidate Nikki Haley...,https://www.msnbc.com/the-reidout/reidout-blog...,"2024 Presidential Election, Nikki Haley, Socia...",Left


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import re

# Adjusting the cleaning function to handle non-string values
def clean_text(text):
    # Check if the text is a string
    if not isinstance(text, str):
        return ""
    
    # Remove HTML tags and non-alphanumeric characters
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Reapply the cleaning function
data['Header_clean'] = data['Header'].apply(clean_text)
data['Content_clean'] = data['Content'].apply(clean_text)
data['Tag_clean'] = data['Page Tags'].apply(clean_text)

# Recheck for missing values
missing_values_updated = data[['Header_clean', 'Content_clean', 'Tag_clean', 'Label']].isnull().sum()

# Display updated missing values
missing_values_updated

Header_clean     0
Content_clean    0
Tag_clean        0
Label            0
dtype: int64

In [23]:
# Creating separate TF-IDF Vectorizers for 'Header_clean' and 'Content_clean'
header_tfidf_vectorizer = TfidfVectorizer(max_features=5000)
content_tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Vectorize 'Header_clean' and 'Content_clean' separately
header_tfidf = header_tfidf_vectorizer.fit_transform(data['Header_clean'])
content_tfidf = content_tfidf_vectorizer.fit_transform(data['Content_clean'])

# Convert to DataFrame for easier handling
header_tfidf_df = pd.DataFrame(header_tfidf.toarray(), columns=header_tfidf_vectorizer.get_feature_names())
content_tfidf_df = pd.DataFrame(content_tfidf.toarray(), columns=content_tfidf_vectorizer.get_feature_names())

# Label Encoding for 'Label' column
label_encoder = LabelEncoder()
data['Label_encoded'] = label_encoder.fit_transform(data['Label'])

# Combine the features and the target
processed_data = pd.concat([header_tfidf_df, content_tfidf_df, data['Label_encoded']], axis=1)

# Display the shape of the processed data
processed_data.shape

(889, 8532)

In [12]:
processed_data.head()

,aaron,abbott,abc,abe,abetting,ability,abortion,abortionad,abortions,about,...,younger,youngkin,your,youre,yourself,youth,youve,zero,zone,Label_encoded
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


### Model Selection

#### Logistic and Random Forest

In [13]:
from sklearn.model_selection import train_test_split

# Splitting the data into features (X) and target (y)
X = processed_data.drop('Label_encoded', axis=1)
y = processed_data['Label_encoded']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Display the shapes of the train and test sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((622, 8531), (267, 8531), (622,), (267,))

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Initializing and training the models
log_reg = LogisticRegression(max_iter=1000)
random_forest = RandomForestClassifier()
log_reg.fit(X_train, y_train)
random_forest.fit(X_train, y_train)

# Evaluating the models
log_reg_pred = log_reg.predict(X_test)
rf_pred = random_forest.predict(X_test)

# Generating classification reports
log_reg_report = classification_report(y_test, log_reg_pred)
rf_report = classification_report(y_test, rf_pred)

print(log_reg_report)
print(rf_report)

              precision    recall  f1-score   support

           0       0.46      0.24      0.32       100
           1       0.33      0.40      0.36        83
           2       0.45      0.62      0.52        84

    accuracy                           0.41       267
   macro avg       0.41      0.42      0.40       267
weighted avg       0.42      0.41      0.39       267

              precision    recall  f1-score   support

           0       0.39      0.20      0.26       100
           1       0.35      0.45      0.39        83
           2       0.33      0.43      0.37        84

    accuracy                           0.35       267
   macro avg       0.36      0.36      0.34       267
weighted avg       0.36      0.35      0.34       267



#### BERT Model

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import normalize
# # Simple preprocessing
# data['text'] = data['Header_clean'] + " " + data['Content_clean']  # Combining header and content
# data = data[['text', 'Label']]  

# # Initialize label encoder
# label_encoder = LabelEncoder()

# # Encode the labels
# data['Label'] = label_encoder.fit_transform(data['Label'])

# # Split the data (with encoded labels)
# train_texts, val_texts, train_labels, val_labels = train_test_split(data['text'], data['Label'], test_size=0.2)

# # Initialize the BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenization function
# def tokenize_function(examples):
#     return tokenizer(examples, padding='max_length', truncation=True, max_length=512)

# # Apply the tokenizer to the dataset
# train_encodings = tokenize_function(train_texts.tolist())
# val_encodings = tokenize_function(val_texts.tolist())

# # Dataset class
# class PoliticalBiasDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# # Convert to dataset format
# train_dataset = PoliticalBiasDataset(train_encodings, train_labels.tolist())
# val_dataset = PoliticalBiasDataset(val_encodings, val_labels.tolist())

# # DataLoader
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# # Load pre-trained BERT model
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data['Label'].unique()))

# # Optimizer and scheduler
# optimizer = AdamW(model.parameters(), lr=5e-5)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3)

# # Move model to GPU
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model.to(device)

# # Training loop
# model.train()
# for epoch in range(3):
#     for batch in train_loader:
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         scheduler.step()

# # Evaluation
# model.eval()
# predictions = []
# references = []
# for batch in val_loader:
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     labels = batch['labels'].to(device)
#     with torch.no_grad():
#         outputs = model(input_ids, attention_mask=attention_mask)
#     logits = outputs.logits
#     predictions.extend(torch.argmax(logits, dim=-1).tolist())
#     references.extend(labels.tolist())

# # Classification report
# print(classification_report(references, predictions))

In [7]:
import matplotlib.pyplot as plt

# Check the distribution of the labels
train_labels.value_counts().plot(kind='bar')
plt.title('Distribution of Classes in Training Set')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.show()

NameError: name 'train_labels' is not defined

In [12]:
# Simple preprocessing
data['text'] = data['Header_clean'] + " " + data['Content_clean'] + " " +  data['Tag_clean']# Combining header and content
data = data[['text', 'Label']]  

In [5]:
import random
import nltk
from nltk.corpus import wordnet

# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

def synonym_replacement(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in nltk.corpus.stopwords.words('english')]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n: # only replace up to n words
            break

    sentence = ' '.join(new_words)
    return sentence

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char.isalpha()])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)


In [37]:
# Step 1: Generate Augmented Text
augmented_texts_5 = [synonym_replacement(sentence, 5) for sentence in data['text']]
augmented_texts_10 = [synonym_replacement(sentence, 10) for sentence in data['text']]
augmented_texts_30 = [synonym_replacement(sentence, 30) for sentence in data['text']]

# Step 2: Create a new DataFrame with augmented text
augmented_data_5 = pd.DataFrame({'text': augmented_texts_5, 'Label': data['Label']})
augmented_data_10 = pd.DataFrame({'text': augmented_texts_10, 'Label': data['Label']})
augmented_data_30 = pd.DataFrame({'text': augmented_texts_30, 'Label': data['Label']})

# Combine the original and augmented datasets
combined_data = pd.concat([data, augmented_data_5, augmented_data_10, augmented_data_30], axis=0)

# Step 3: Shuffle the combined dataset
combined_data = combined_data.sample(frac=1).reset_index(drop=True)

In [38]:
combined_data

,text,Label
0,hey liberal media did you miss this part about...,Right
1,the us is expire to have to raise tax the us g...,Center
2,there is no first amendment right to overturn ...,Left
3,alfrednobel prize awarded to duo who made rapi...,Left
4,report irs removes entire team probing hunter ...,Right
...,...,...
3551,rudy giuliani pleads not shamed in georgia ele...,Left
3552,armyintelligence whitethorn interrupt math and...,Center
3553,gaza hospital explosion poses questions about ...,Center
3554,a year old grandmother fights back after gover...,Center


In [12]:
from sklearn.preprocessing import normalize
x = label_encoder.fit_transform(data['Label'])
dd = normalize(x[:,np.newaxis], axis=0).ravel()
dd.fit_

array([0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581989,
       0.02581989, 0.02581989, 0.02581989, 0.02581989, 0.02581

In [31]:
dd.shape

(3556,)

In [29]:
x.shape

(3556,)

In [14]:
torch.tensor(dd[1])

tensor(0.0258, dtype=torch.float64)

In [14]:
data['Label'].replace({"Left": 0.0, "Center": 0.5, "Right": 1.0}, inplace = True)

In [15]:
data['Label']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
884    1.0
885    1.0
886    1.0
887    1.0
888    1.0
Name: Label, Length: 889, dtype: float64

In [6]:
label_encoder = LabelEncoder()
data['Label'].replace({"Left": 0.0, "Center": 0.5, "Right": 1.0}, inplace = True)
data['Label'] = label_encoder.fit_transform(data['Label'])

In [7]:
data['Label']

0      0
1      0
2      0
3      0
4      0
      ..
884    2
885    2
886    2
887    2
888    2
Name: Label, Length: 889, dtype: int64

In [13]:
# Try with larger dataset
# data = combined_data

# Initialize label encoder
label_encoder = LabelEncoder()

# Encode the labels
# x = label_encoder.fit_transform(data['Label'])
# data['Label'] = normalize(x[:,np.newaxis], axis=0).ravel()
data['Label'] = label_encoder.fit_transform(data['Label'])
# data['Label'].replace({"Left": 0.0, "Center": 0.5, "Right": 1.0}, inplace = True)
# data['Label'] = label_encoder.fit_transform(data['Label'])

# Split the data (with encoded labels)
train_texts, val_texts, train_labels, val_labels = train_test_split(data['text'], data['Label'], test_size=0.2)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples, padding='max_length', truncation=True, max_length=512)

# Apply the tokenizer to the dataset
train_encodings = tokenize_function(train_texts.tolist())
val_encodings = tokenize_function(val_texts.tolist())

# Dataset class
class PoliticalBiasDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convert to dataset format
train_dataset = PoliticalBiasDataset(train_encodings, train_labels.tolist())
val_dataset = PoliticalBiasDataset(val_encodings, val_labels.tolist())

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data['Label'].unique()))

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-6)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3)

# Move model to GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Training loop
model.train()
for epoch in range(3):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        print(labels.shape)
        print(input_ids.shape)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

# Evaluation
model.eval()
predictions = []
references = []
for batch in val_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions.extend(torch.argmax(logits, dim=-1).tolist())
    references.extend(labels.tolist())

# Classification report
print(classification_report(references, predictions))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yic055/.local/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
torch.Size([8])
torch.Size([8, 512])
t

KeyboardInterrupt: 

- lr: 0.00001 -->0.69
- lr: 0.00005 --> 0.41 
- lr: 0.0001 --> 0.35
- lr: 0.0005 --> 0.34

# Liar Classification

In [ ]:
train2.tsv
val2.tsv
test2.tsv

In [24]:
pd.read_csv("train2.tsv",sep='\t', header=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,1.0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe..."
2,2.0,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...
3,3.0,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...
4,4.0,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10237,10264.0,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report""",They compounded their error by combining full ...
10238,10265.0,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview,"Romney said that ""Obamacare means that for up..."
10239,10266.0,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate,But that it leaves out important details and t...
10240,10267.0,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...,"We checked the research and, quite frankly, fi..."


In [27]:
import requests
from bs4 import BeautifulSoup

# URL of the Politifact fact-checks page
url = "https://www.politifact.com/factchecks/list/?ruling=true"

def scrape_politifact(url):
    # Send a request to the website
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all fact-checks on the page
    fact_checks = soup.find_all('li', class_='o-listicle__item')

    data = []
    for fact_check in fact_checks:
        # Extract the statement and label
        statement = fact_check.find('div', class_='m-statement__quote').get_text(strip=True)
        label = fact_check.find('div', class_='m-statement__meter').find('img')['alt']

        # Extract the link to the detailed article
        link = fact_check.find('div', class_='m-statement__quote').find('a')['href']
        detailed_response = requests.get("https://www.politifact.com" + link)
        detailed_soup = BeautifulSoup(detailed_response.content, 'html.parser')

        # Extract the summary from the detailed article
        summary_div = detailed_soup.find('div', class_='short-on-time')
        
        if summary_div:
            summary = summary_div.get_text(strip=True)
            data.append({
                'statement': statement,
                'label': label,
                'justification': summary
            })

    return data

In [28]:
urls = ["https://www.politifact.com/factchecks/list/?page=2&ruling=true",
        "https://www.politifact.com/factchecks/list/?page=3&ruling=true", 
        "https://www.politifact.com/factchecks/list/?page=4&ruling=true", 
        "https://www.politifact.com/factchecks/list/?page=5&ruling=true",
        "https://www.politifact.com/factchecks/list/?ruling=mostly-true", 
        "https://www.politifact.com/factchecks/list/?page=2&ruling=mostly-true",
        "https://www.politifact.com/factchecks/list/?page=3&ruling=mostly-true", 
        "https://www.politifact.com/factchecks/list/?page=4&ruling=mostly-true", 
        "https://www.politifact.com/factchecks/list/?page=5&ruling=mostly-true",
        "https://www.politifact.com/factchecks/list/?ruling=half-true",
        "https://www.politifact.com/factchecks/list/?page=2&ruling=half-true", 
        "https://www.politifact.com/factchecks/list/?page=3&ruling=half-true", 
        "https://www.politifact.com/factchecks/list/?page=4&ruling=half-true", 
        "https://www.politifact.com/factchecks/list/?page=5&ruling=half-true",
        "https://www.politifact.com/factchecks/list/?ruling=barely-true", 
        "https://www.politifact.com/factchecks/list/?page=2&ruling=barely-true", 
        "https://www.politifact.com/factchecks/list/?page=3&ruling=barely-true", 
        "https://www.politifact.com/factchecks/list/?page=4&ruling=barely-true", 
        "https://www.politifact.com/factchecks/list/?page=5&ruling=barely-true", 
        "https://www.politifact.com/factchecks/list/?ruling=false", 
        "https://www.politifact.com/factchecks/list/?page=2&ruling=false", 
        "https://www.politifact.com/factchecks/list/?page=3&ruling=false", 
        "https://www.politifact.com/factchecks/list/?page=4&ruling=false", 
        "https://www.politifact.com/factchecks/list/?page=5&ruling=false",
        "https://www.politifact.com/factchecks/list/?ruling=pants-fire", 
        "https://www.politifact.com/factchecks/list/?page=2&ruling=pants-fire", 
        "https://www.politifact.com/factchecks/list/?page=3&ruling=pants-fire", 
        "https://www.politifact.com/factchecks/list/?page=4&ruling=pants-fire", 
        "https://www.politifact.com/factchecks/list/?page=5&ruling=pants-fire"]

In [29]:
scraped_data = scrape_politifact(url)

for url in urls:
    additional_data = scrape_politifact(url)
    scraped_data.extend(additional_data)

In [30]:
def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['statement', 'label', 'justification'])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [31]:
import csv
save_to_csv(scraped_data, 'politifact_plus_data.csv')

In [32]:
pd.read_csv("politifact_plus_data.csv")

,statement,label,justification
0,“We sell arms to 60% of the world's autocrats....,true,"The U.S. is the world’s largest arms exporter,..."
1,“We produce a lot more wind energy in Texas th...,true,"Texas produced more than 114,000 megawatt hour..."
2,"Potato chips, KitKat bars and Viagra are not t...",true,Grocery items including potato chips and KitKa...
3,"“Since 1981, the state Senate has only rejecte...",true,The state Senate voted to fire eight of the go...
4,“Lego donates model MRI kits to hospitals to h...,true,A Lego employee and physicians at a Danish hos...
...,...,...,...
884,Chemtrails are being put into the atmosphere a...,pants-fire,Atmospheric chemists and geochemists dispute t...
885,"""There is no Biden presidency. The real Biden ...",pants-fire,President Joe Biden and former Secretary of St...
886,Video shows someone impersonating Joe Biden in...,pants-fire,"If this video shows anything, it’s that Biden,..."
887,“El CEO de Pfizer es un lagarto (demonio)”.,pants-fire,"El CEO de Pfizer, Albert Bourla, es humano.La ..."
